In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
import matplotlib.image as mpimg
from matplotlib import pyplot as plt
import seaborn as sns
import albumentations as A
from skimage.exposure import rescale_intensity


import numpy as np
import matplotlib.pyplot as plt

from skimage import data
from skimage.color import rgb2hed, hed2rgb

Declaration of a simple preprocessor function.  Preprocessor functions are used to preprocess an instance before labeling it. We use @preprocessor(name,resources) decorator to declare a function as preprocessor.

In [ ]:
from spear.labeling import preprocessor

@preprocessor()
def imagePreProcessing(df):

        img = cv2.imread(df)
        #Noise Removing
        image = cv2.fastNlMeansDenoisingColored(img,None,10,10,7,21)
        #Gaussian Blur
        gaussian_3 = cv2.GaussianBlur(image, (9,9), 10.0) #unblur
        image = cv2.addWeighted(image, 1.5, gaussian_3, -0.5, 0, image)
        #Laplacian Filter
        #kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]]) #filter
        #image = cv2.filter2D(image, -1, kernel)
        return image

In [ ]:
from spear.labeling import labeling_function

In [ ]:
@roi_selection(pre=[imagePreProcessing])
def LF1(c):
    # input image
    ihc_rgb = c

    # Separate the stains from the IHC image
    ihc_hed = rgb2hed(ihc_rgb)

    # Create an RGB image for each of the stains
    null = np.zeros_like(ihc_hed[:, :, 0])
    ihc_h = hed2rgb(np.stack((ihc_hed[:, :, 0], null, null), axis=-1))
    ihc_h = ihc_h*255      
    image1 = ihc_h.astype('uint8')
   
    # cv2.cvtColor is applied over the
    # image input with applied parameters
    # to convert the image in grayscale 
    img = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    
    # Apply GaussianBlur to reduce image noise if it is required
    blur = cv2.GaussianBlur(img,(5,5),0)
    # Otsu's thresholding after Gaussian filtering
    otsu_threshold, image_result = cv2.threshold(
         blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,)
    # Invert the image
    thresh2 = 255-image_result
    # Get connected components
    output = cv2.connectedComponentsWithStats(thresh2)
    (numLabels, labels, stats, centroids) = output
    mask = np.zeros(thresh2.shape, dtype="uint8")
    # Iterate over the number of unique connected component labels, skipping
    # over the first label (as label zero is the background)
    for i in range(1, numLabels):
        # extract the connected component statistics for the current label
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        # ensure the width, height, and area are all neither too small
        # nor too big
        keepWidth = w > 20
        keepHeight = h > 20
        keepArea = 100 < area < 2000
        # ensure the connected component we are examining passes all
        # three tests
        if all((keepWidth, keepHeight, keepArea)):
            # construct a mask for the current connected component and
            # then take the bitwise OR with the mask
            componentMask = (labels == i).astype("uint8") * 1
            mask = cv2.bitwise_or(mask, componentMask)
    # Find contours of the mask        
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    bounding_boxes = []
    # Iterate through the contours and find bounding boxes
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x+w, y+h])
    return bounding_boxes

In [ ]:
@roi_selection(pre=[imagePreProcessing])
def LF2(c):
    ihc_rgb = c

    # Separate the stains from the IHC image
    ihc_hed = rgb2hed(ihc_rgb)

    # Create an RGB image for each of the stains
    null = np.zeros_like(ihc_hed[:, :, 0])
    ihc_d = hed2rgb(np.stack((ihc_hed[:, :, 2], null, null), axis=-1))

    ihc_d = ihc_d*255      

   

    image1 = ihc_d.astype('uint8')
    # cv2.cvtColor is applied over the
    # image input with applied parameters
    # to convert the image in grayscale 
    img = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
    
    blur = cv2.GaussianBlur(img,(5,5),0)
    otsu_threshold, image_result = cv2.threshold(
         blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,)
    thresh2 = 255-image_result
    output = cv2.connectedComponentsWithStats(thresh2)
    (numLabels, labels, stats, centroids) = output
    mask = np.zeros(thresh2.shape, dtype="uint8")
    for i in range(1, numLabels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        keepWidth = w > 20
        keepHeight = h > 20
        keepArea = 100 < area < 2000
        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 1
            mask = cv2.bitwise_or(mask, componentMask)
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x+w, y+h])
    return bounding_boxes

In [ ]:
@labeling_function(pre=[imagePreProcessing])
def LF3(c):
    ihc_rgb = c

    # Separate the stains from the IHC image
    ihc_hed = rgb2hed(ihc_rgb)

    # Create an RGB image for each of the stains
    null = np.zeros_like(ihc_hed[:, :, 0])
    # Rescale hematoxylin and DAB channels and give them a fluorescence look
    h = rescale_intensity(ihc_hed[:, :, 0], out_range=(0, 1),
                      in_range=(0, np.percentile(ihc_hed[:, :, 0], 99)))
    d = rescale_intensity(ihc_hed[:, :, 2], out_range=(0, 1),
                      in_range=(0, np.percentile(ihc_hed[:, :, 2], 99)))
        

    # Cast the two channels into an RGB image, as the blue and green channels respectively
    zdh = np.dstack((null, d, h))

    zdh = zdh*255      


    image1 = zdh.astype('uint8')
   
    # cv2.cvtColor is applied over the
    # image input with applied parameters
    # to convert the image in grayscale 
    img = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)

    
    blur = cv2.GaussianBlur(img,(5,5),0)
    otsu_threshold, image_result = cv2.threshold(
         blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,)
    thresh2 = 255-image_result
    output = cv2.connectedComponentsWithStats(thresh2)
    (numLabels, labels, stats, centroids) = output
    mask = np.zeros(thresh2.shape, dtype="uint8")
    for i in range(1, numLabels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        keepWidth = w > 20
        keepHeight = h > 20
        keepArea = 100 < area < 2000
        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 1
            mask = cv2.bitwise_or(mask, componentMask)
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x+w, y+h])
    return bounding_boxes

In [ ]:
@labeling_function(pre=[imagePreProcessing])
def LF4(c):
    image = c
    # Split
    red = img[:, :, 0]

    image_result = red*255
    
    img = image_result
    blur = cv2.GaussianBlur(img,(5,5),0)
    otsu_threshold, image_result = cv2.threshold(
         blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,)
    thresh2 = 255-image_result
    output = cv2.connectedComponentsWithStats(thresh2)
    (numLabels, labels, stats, centroids) = output
    mask = np.zeros(thresh2.shape, dtype="uint8")
    for i in range(1, numLabels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        keepWidth = w > 20
        keepHeight = h > 20
        keepArea = 100 < area < 2000
        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 1
            mask = cv2.bitwise_or(mask, componentMask)
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x+w, y+h])
    return bounding_boxes

In [ ]:
@labeling_function(pre=[imagePreProcessing])
def LF5(c):
    image = c
    # Split
    green = img[:, :, 1]

    image_result = green*255
    
    img = image_result
    blur = cv2.GaussianBlur(img,(5,5),0)
    otsu_threshold, image_result = cv2.threshold(
         blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,)
    thresh2 = 255-image_result
    output = cv2.connectedComponentsWithStats(thresh2)
    (numLabels, labels, stats, centroids) = output
    mask = np.zeros(thresh2.shape, dtype="uint8")
    for i in range(1, numLabels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        keepWidth = w > 20
        keepHeight = h > 20
        keepArea = 100 < area < 2000
        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 1
            mask = cv2.bitwise_or(mask, componentMask)
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x+w, y+h])
    return bounding_boxes

In [ ]:
@labeling_function(pre=[imagePreProcessing])
def LF6(c):
    image = c
    # Split
    blue = img[:, :, 2]

    image_result = blue*255
    
    img = image_result
    blur = cv2.GaussianBlur(img,(5,5),0)
    otsu_threshold, image_result = cv2.threshold(
         blur, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU,)
    thresh2 = 255-image_result
    output = cv2.connectedComponentsWithStats(thresh2)
    (numLabels, labels, stats, centroids) = output
    mask = np.zeros(thresh2.shape, dtype="uint8")
    for i in range(1, numLabels):
        x = stats[i, cv2.CC_STAT_LEFT]
        y = stats[i, cv2.CC_STAT_TOP]
        w = stats[i, cv2.CC_STAT_WIDTH]
        h = stats[i, cv2.CC_STAT_HEIGHT]
        area = stats[i, cv2.CC_STAT_AREA]
        keepWidth = w > 20
        keepHeight = h > 20
        keepArea = 100 < area < 2000
        if all((keepWidth, keepHeight, keepArea)):
            componentMask = (labels == i).astype("uint8") * 1
            mask = cv2.bitwise_or(mask, componentMask)
    contours = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    bounding_boxes = []
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        bounding_boxes.append([x, y, x+w, y+h])
    return bounding_boxes